In [1]:
import sys, os

MARIPOSA_PATH = "/home/yizhou7/mariposa/"

sys.path.append(MARIPOSA_PATH + "src")
os.chdir(MARIPOSA_PATH)

from debugger.debugger import Debugger3
from debugger.tree_node import NodeRef
from evaluator import Evaluator
from tabulate import tabulate
from tqdm import tqdm
import numpy as np
import networkx as nx

In [2]:
q = "data/projs/bench_unstable/base.z3/d_lvbkv--ByteBlockCacheSystem-ByteSystem.i.dfy.CheckWellformed__ByteSystem.__default.DiskInternalStepPreservesInv.smt2"
# dbg = Debugger3(q)
r = Evaluator(q)
# print(r.status)
# r.collect_garbage()
report = r.build_report()

[INFO] [init] dbg root: dbg/2ead327dad/ 


In [3]:
editor = r.editor

[DEBUG] [edit] proof path: dbg/2ead327dad/proofs/reseed.16227469897767659683.proof 
[DEBUG] [edit] trace path: dbg/2ead327dad/traces/rename.8473661985171767378 


782b9dc8e4f875512b508dcef877c04e.pickle
[WARN] [differ] qid constructor_accessor_axiom not found in dbg/2ead327dad/orig.smt2 
[WARN] [differ] qid <null> not found in dbg/2ead327dad/orig.smt2 


In [4]:
# print the instantiation report
extended = report.freq.merge(report.tested, on="qname")
extended["stabilized"] = extended["qname"].isin(report.stabilized["qname"])
extended = extended.sort_values(by="trace_count", ascending=False)
extended = extended[["qname", "trace_count", "stabilized", "action", "time", "result", "edit_path"]]
print(tabulate(extended, headers='keys', tablefmt='psql'))

+------+--------------------+---------------+--------------+--------------+--------+----------+-------------------------------------------------------+
|      | qname              |   trace_count | stabilized   | action       |   time | result   | edit_path                                             |
|------+--------------------+---------------+--------------+--------------+--------+----------+-------------------------------------------------------|
|  235 | mariposa_qid_390   |         16642 | False        | erase        |  1.311 | unsat    | data/projs/singleton_2ead327dad/base.z3/43fc9ccb.smt2 |
|  710 | mariposa_qid_13751 |          4202 | False        | erase        | 60     | timeout  | data/projs/singleton_2ead327dad/base.z3/0df06d19.smt2 |
|  589 | mariposa_qid_70    |          2231 | False        | inst_keep    | 60     | timeout  | data/projs/singleton_2ead327dad/base.z3/30216e7a.smt2 |
| 1122 | mariposa_qid_121   |          2220 | False        | erase        |  1.384 | uns

In [5]:
# print the tested report
sorted_tested = report.tested.sort_values(by=["time"])
print(tabulate(sorted_tested, headers='keys', tablefmt='psql'))

+------+--------------------+--------------+----------+--------+-------------------------------------------------------+
|      | qname              | action       | result   |   time | edit_path                                             |
|------+--------------------+--------------+----------+--------+-------------------------------------------------------|
|  170 | mariposa_qid_13423 | erase        | unsat    |  1.079 | data/projs/singleton_2ead327dad/base.z3/25a8266e.smt2 |
| 1088 | mariposa_qid_13670 | erase        | unsat    |  1.083 | data/projs/singleton_2ead327dad/base.z3/ef86e617.smt2 |
|  532 | mariposa_qid_13974 | erase        | unsat    |  1.086 | data/projs/singleton_2ead327dad/base.z3/7d696275.smt2 |
|  446 | mariposa_qid_11096 | skolemize    | unsat    |  1.088 | data/projs/singleton_2ead327dad/base.z3/690fabed.smt2 |
|  219 | mariposa_qid_7170  | erase        | unsat    |  1.089 | data/projs/singleton_2ead327dad/base.z3/3053963e.smt2 |
|  368 | mariposa_qid_13984 | er

In [6]:
print(tabulate(report.stabilized.sort_values(by="qname"), headers='keys', tablefmt='psql'))

+----+--------------------+----------+-------------------------------------------------------+
|    | qname              | action   | edit_path                                             |
|----+--------------------+----------+-------------------------------------------------------|
|  1 | mariposa_qid_13974 | erase    | data/projs/singleton_2ead327dad/base.z3/7d696275.smt2 |
|  0 | mariposa_qid_14025 | erase    | data/projs/singleton_2ead327dad/base.z3/6eabf1b8.smt2 |
+----+--------------------+----------+-------------------------------------------------------+


In [7]:
pa = editor.proof

In [11]:
from debugger.tree_node import AppNode

qii = pa.get_inst_info_under_qname("mariposa_qid_2111")

for i in qii.get_all_insts():
    # print(i)
    # print(pa.dump_node(i))
    children = set()
    if not pa.has_node(i):
        node = pa.lookup_node(i)
        assert isinstance(node, AppNode)
        assert node.name == "or"
        for c in node.children:
            children.add(c)
    else:
        children.add(i)

    reachable = set()
    for c in children:
        reachable.add(c)
        for j in nx.descendants(pa, c):
            reachable.add(j)
    
    ref_counts = dict()
    for j in reachable:
        ref_count = pa.in_degree(j)
        ref_counts[j] = ref_count
    
    ref_counts = sorted(ref_counts.items(), key=lambda x: x[1], reverse=True)

    for j, ref_count in ref_counts:
        print(ref_count, pa.dump_node(j))


40 DatatypeTypeType
26 |step#0@@171|
10 (AsyncDisk.InternalStep.HavocConflictingWritesStep_q |step#0@@171|)
10 (not (= (type |step#0@@171|) DatatypeTypeType))
9 (AsyncDisk.InternalStep.ProcessReadFailureStep_q |step#0@@171|)
8 (AsyncDisk.InternalStep.ProcessWriteStep_q |step#0@@171|)
5 (= (type |step#0@@171|) DatatypeTypeType)
5 (AsyncDisk.InternalStep.HavocConflictingWriteReadStep_q |step#0@@171|)
5 (|$IsA#AsyncDisk.InternalStep| |step#0@@171|)
1 (not (|$IsA#AsyncDisk.InternalStep| |step#0@@171|))
1 (type |step#0@@171|)


In [12]:
from debugger.inst_graph import TraceInstGraph

trace = editor.trace
g = trace.build_inst_graph()

[INFO] [graph] building dbg/2ead327dad/graphs/rename.8473661985171767378.txt 
[INFO] [stats] building dbg/2ead327dad/stats/rename.8473661985171767378.txt 


In [13]:
ratios = g.compute_sub_ratios({"mariposa_qid_14025"})
for qidx, ratio in ratios.items():
    print(qidx, ratio * g.blames[qidx].blamed_count)

43594 1
26048 1.0
26051 1.0
26078 1.0
42675 1.0
42905 1.0
35016 28.923067224145903
35017 29.608219112601617
35021 48.563986591046046
43553 1.0
43557 2.0
43572 1.0
43574 1.0
43650 194.78127193472568
43728 973.4958489434367
43729 46.35749361068699
43730 46.52030857378685
43929 210.63846803805805
44133 73.61730451356723
44135 211.78005347401023
44168 84.61181873245368
44530 17.924618357214545
44551 3.86278143879712
44600 4.045677664089813
44625 627.3224711803821
44626 318.21220968981584
44627 318.21220968981584
44650 493.7070496615455
44654 176.59678817097927
44658 176.5967881709792
36479 4.982853772245759
36482 1546.595485003386
36483 1379.116709993751
44683 29.126171504258046
44684 29.126171504258046
44691 86.49624727822835
36544 23.665014347209663
36545 23.665014347209663
36609 6.0
36610 6.0
36612 6.0
4292 24.18785470505843
4311 36.922847743638485
4325 10.998219298020862
4327 10.998219298020862
45637 21.844628628193533
45638 21.844628628193533
45645 21.999527948093288
45692 185.2820338

In [ ]:
ratios = dict()
for root_name in tqdm(editor.list_qnames(root_only=True)):
    t_group = trace.get_group_stat(root_name)
    ratios[root_name] = g.compute_sub_ratios(t_group.keys())


100%|██████████| 11942/11942 [03:30<00:00, 56.77it/s] 


In [17]:
scores = dict()
for root_name in ratios.keys():
    rs = ratios[root_name]
    score = 0
    for qidx, ratio in rs.items():
        score += ratio * g.blames[qidx].blamed_count
    scores[root_name] = score

scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)
for root_name, score in scores:
    print(root_name, score)

mariposa_qid_1672 393578.3849533169
mariposa_qid_14025 386186.94073471223
mariposa_qid_390 334864.3684661526
mariposa_qid_13974 235826.50583308045
mariposa_qid_14001 146437.55709331625
mariposa_qid_12432 111973.1280189032
mariposa_qid_12427 99903.37729629179
mariposa_qid_14201 76269.90717144051
mariposa_qid_13780 75556.42083974864
mariposa_qid_13493 73245.110213719
mariposa_qid_13970 72951.51908305673
mariposa_qid_1662 72492.11164178775
mariposa_qid_13431 66387.94935911913
mariposa_qid_14203 59442.624673915656
mariposa_qid_13487 53049.713297334754
mariposa_qid_12439 52565.289319874886
mariposa_qid_13425 51862.94167307987
mariposa_qid_11787 51661.15520901153
mariposa_qid_13776 50540.68774185779
mariposa_qid_13424 50471.94312895039
mariposa_qid_363 49611.557696267635
mariposa_qid_13457 49358.247350150355
mariposa_qid_13765 49230.80971703776
mariposa_qid_13813 48523.012722037805
mariposa_qid_13523 48523.012722037805
mariposa_qid_388 48520.78821475417
mariposa_qid_11834 47103.72578856122
m